In [ ]:
import json
import os
import pandas as pd
from firecloud import api as fapi
from plotnine import *
from pprint import pprint

In [ ]:
# define datasets with format {"label": "submission_id"} 
# where submission_id corresponds to the run of GatkConcordanceValidation in this workspace
datasets = {
    'chr1 split': '017c9e93-29d5-41f5-9490-c18fe0c3b1f7',
    'chr1 no split': '300d7b27-955a-486c-bf4e-4c79677fb5cf'}

sample_to_ancestry_tsv = 'gs://fc-secure-070f3cd7-c0bc-4125-90e4-7962a696cc72/input_files/evaluation/hgdp_951_sample_to_ancestry.tsv'

In [ ]:
WORKSPACE = os.getenv('WORKSPACE_NAME')
PROJECT = os.getenv('WORKSPACE_NAMESPACE')
WORKFLOW_NAME = 'GatkConcordanceValidation'

def get_outputs_from_submission(submission_id):
    """Extract the tsv output from a submission, assuming only one workflow in the submission"""
    submission = fapi.get_submission(PROJECT, WORKSPACE, submission_id).json()
    workflow_id = submission['workflows'][0]['workflowId']
    
    outputs = fapi.get_workflow_outputs(PROJECT, WORKSPACE, submission_id, workflow_id).json()
    return outputs['tasks'][WORKFLOW_NAME]['outputs']

In [ ]:
data = pd.DataFrame()
for label, submission_id in datasets.items():
    # get outputs
    outputs = get_outputs_from_submission(submission_id)
    # extract correlation tsv
    correlation_tsv = outputs['GatkConcordanceValidation.combined_correlations']
    df = pd.read_csv(correlation_tsv, sep="\t", comment="#", dtype={'INDEL_CORRELATION':float,
                                                     'SNP_CORRELATION':float}, na_values="?")
    df['panel']=label
    data = pd.concat([data, df], ignore_index=True)

In [ ]:
# display first few lines of specified section
chrom_to_preview = 'chr1'
data[data['CHROMOSOME']==chrom_to_preview].head()

In [ ]:
# OPTIONAL - remove certain samples
remove_samples = False

if remove_samples:
    samples_to_remove = ["HGDP00001"]
    
    print(f"found {samples_to_remove[0]} in data: {samples_to_remove[0] in data['SAMPLE'].tolist()}")
    
    # remove
    data = data[~data.SAMPLE.isin(samples_to_remove)]
    
    # confirm
    print(f"found {samples_to_remove[0]} in data: {samples_to_remove[0] in data['SAMPLE'].tolist()}")
else:
    print("not removing any samples")

In [ ]:
data['r2_snp']=data['SNP_CORRELATION']**2
data['r2_indel']=data['INDEL_CORRELATION']**2

In [ ]:
ancestry = pd.read_csv(sample_to_ancestry_tsv,sep="\t", header=None, names=["sample", "ancestry"])
data = data.merge(ancestry, left_on="SAMPLE", right_on="sample")

In [ ]:
data.head()

In [ ]:
summary_by_contig = (data.query('CHROMOSOME!="WholeGenome"')
                         .groupby(['BIN_CENTER',
                                   'panel',
                                   'CHROMOSOME']).
                          agg(r2_snp = pd.NamedAgg('r2_snp','mean'),
                              r2_indel = pd.NamedAgg('r2_indel','mean')
                             )
                         )
summary_by_contig.head(3)

In [ ]:
summary_by_contig_and_ancestry = (data.query('CHROMOSOME!="WholeGenome"').groupby(['BIN_CENTER','panel',
                                                       'CHROMOSOME','ancestry']).
                                  agg(r2_snp = pd.NamedAgg('r2_snp','mean'),
                                      r2_indel = pd.NamedAgg('r2_indel','mean')
                                     )
                                 )
summary_by_contig_and_ancestry.head(3)

In [ ]:
summary_by_ancestry = (data.query('CHROMOSOME=="WholeGenome"').groupby(['BIN_CENTER','panel','ancestry']).
                                  agg(r2_snp = pd.NamedAgg('r2_snp','mean'),
                                      r2_indel = pd.NamedAgg('r2_indel','mean')
                                     )
                                 )

summary_by_ancestry.head(3)

In [ ]:
# plot SNP mean r-squared for all panels, whole genome, split by ancestry
(
    ggplot(summary_by_ancestry.reset_index(), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("ancestry") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab("Mean r$^2$ for SNPs")
)

In [ ]:
# plot INDEL mean r-squared for all panels, whole genome, split by ancestry
(
    ggplot(summary_by_ancestry.reset_index(), aes(x='BIN_CENTER', y='r2_indel', color='panel')) +
    geom_point() + geom_line() + facet_wrap("ancestry") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab("Mean r$^2$ for INDELs")
)

In [ ]:
# plot SNP mean r-squared for all panels, split by chromosome
(
    ggplot(summary_by_contig.reset_index(), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab("Mean r$^2$ for SNPs")
)

In [ ]:
# plot INDEL mean r-squared for all panels, split by chromosome
(
    ggplot(summary_by_contig.reset_index(), aes(x='BIN_CENTER', y='r2_indel', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab("Mean r$^2$ for INDELs")
)

In [ ]:
# show all ancestries in the data
data['ancestry'].unique()

In [ ]:
# sas
ancestry_key = 'sas'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# afr
ancestry_key = 'afr'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# oth
ancestry_key = 'oth'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# nfe
ancestry_key = 'nfe'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# mid
ancestry_key = 'mid'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# amr
ancestry_key = 'amr'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# eas
ancestry_key = 'eas'
(
    ggplot(summary_by_contig_and_ancestry.reset_index().query(f'ancestry=="{ancestry_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='panel')) +
    geom_point() + geom_line() + facet_wrap("CHROMOSOME") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs for {ancestry_key} samples")
)

In [ ]:
# select a panel, plot SNPs
panel_key = 'chr1 split'
(
    ggplot(both_r2_summary_by_contig.reset_index().query(f'panel=="{panel_key}"'), aes(x='BIN_CENTER', y='r2_snp', color='factor(contig)')) +
    geom_point() + geom_line() + facet_wrap("ancestry") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for SNPs ({panel_key})")
)

In [ ]:
# plot INDELs for selected panel
(
    ggplot(both_r2_summary_by_contig.reset_index().query(f'panel=="{panel_key}"'), aes(x='BIN_CENTER', y='r2_indel', color='factor(contig)')) +
    geom_point() + geom_line() + facet_wrap("ancestry") + theme_bw() + scale_x_log10() + ylim(0,1) +
    xlab("Allele Frequency") + ylab(f"Mean r$^2$ for INDELs ({panel_key})")
)

In [ ]:
# example query
summary_by_contig_and_ancestry.query('ancestry=="afr" and CHROMOSOME=="chr3"')